In [1]:
%pylab inline

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd

In [3]:
koubei = pd.read_csv(
    'data/ijcai2016_koubei_train', header=None, names=['user_id', 'merchant_id', 'location_id', 'timestamp']
)

In [9]:
pred = pd.read_csv('data/ijcai2016_koubei_test', header=None, names=['user_id', 'location_id'])

In [14]:
merchant = pd.read_csv('data/ijcai2016_merchant_info', header=None, names=['merchant_id', 'budget', 'locations'])

# 生成测试数据

这里只取“预测集”里的和之前月份取交集的用户作为测试。因为给定的集合实际上由**线上、线下**两部分组成，因此，很难组合出和实际预测集相近的验证集。这样的话，虽然和实际情况并不相符，但是可以理解整个计算过程。

取 10 月份的口碑记录，作为假想的预测集。

In [7]:
koubei10 = koubei[koubei.timestamp.between(20151001, 20151031)]

取 11 月份的口碑记录，作为预测集对照的实际集。

In [5]:
koubei11 = koubei[koubei.timestamp.between(20151101, 20151131)]

统计相关的数据。

In [11]:
print('Oct. no. of users: %d' % (koubei10.user_id.nunique()))
print('Nov. no. of users: %d' % (koubei11.user_id.nunique()))
print('Old users in Nov.: %d (%.2f%%)' % (
    koubei11[koubei11.user_id.isin(koubei10.user_id)].user_id.nunique(),
    koubei11[koubei11.user_id.isin(koubei10.user_id)].user_id.nunique() / float(koubei11.user_id.nunique()) * 100
))

Oct. no. of users: 116413
Nov. no. of users: 158544
Old users in Nov.: 67522 (42.59%)


与实际数据对比。

In [12]:
print('Dec. no. of users: %d' % pred.user_id.nunique())
print('Nov. no. of users: %d' % koubei11.user_id.nunique())
print('Old users in Dec.: %d (%.2f%%)' % (
    pred[pred.user_id.isin(koubei11.user_id)].user_id.nunique(),
    pred[pred.user_id.isin(koubei11.user_id)].user_id.nunique() / float(pred.user_id.nunique()) * 100
))

Dec. no. of users: 465366
Nov. no. of users: 158544
Old users in Dec.: 37838 (8.13%)


这里存在两个假设：

1. 给定的预测集里并非所有用户都有购买行为；
2. 给定的训练集的样本太小。

计算 10 月预测 11 月的 F1 值。

In [84]:
def evaluate(pset, aset):
    ''' 
    pset is the predicted set.
    aset is the actual set.
    '''
    pset = pset[['user_id', 'location_id', 'merchant_id']].drop_duplicates()
    aset = aset[['user_id', 'location_id', 'merchant_id']].drop_duplicates()
    pset = pd.merge(pset, aset[['user_id', 'location_id']], on=['user_id', 'location_id'], how='inner')
    hit = pd.merge(aset, pset, on=['user_id', 'location_id', 'merchant_id'], how="inner").drop_duplicates()
    a = pd.merge(
        merchant[['merchant_id', 'budget']],
        hit.groupby('merchant_id').user_id.count().reset_index(),
        on='merchant_id', how='inner'
    ).apply(lambda d: min(d['budget'], d['user_id']), axis=1).sum()
    b = pd.merge(
        merchant[['merchant_id', 'budget']],
        aset.groupby('merchant_id').user_id.count().reset_index(),
        on='merchant_id', how='inner'
    ).apply(lambda d: min(d['budget'], d['user_id']), axis=1).sum()
    c = pset.groupby('merchant_id').user_id.count().sum()
    print a, b, c
    p = a / float(c)
    r = a / float(b)
    print('Predict: %.2f%%' % (p * 100))
    print('Recall: %.2f%%'  % (r * 100))
    return 2 * p * r / (p + r)

测试 10 月预测 11 月的测评（纯口碑数据）。

In [85]:
evaluate(koubei10, koubei11)

52274 161186 110218
Predict: 47.43%
Recall: 32.43%


0.3852117139025217

In [86]:
evaluate(
    koubei[koubei.timestamp.between(20150701, 20151031)],
    koubei11
)

57818 161186 130119
Predict: 44.43%
Recall: 35.87%


0.3969585142719829

基本可以得出两个结论：

1. 给定的训练集的样本太小，基本没有淘宝中转化的 → 要挖掘这部分用户依靠这部分数据的作用可能很有限；
2. 预测准确率并没有以前想的那么高，只有大概 60% 。

由此可以推出之前的**预测准确率并没有那么高，反而召回率比较高**，因此，增加热门地点搜索以后对拉高召回率帮助不大，而准确率上下降太多了。

从预测的 F1 值反推，目前可能的方案是：

Prediction | Recall | F1
-----------|--------|-----
0.1 | 0.1 | 0.1
0.1 | 0.2 | 0.133
0.1 | 0.3 | 0.15
0.1 | 0.4 | 0.160
0.1 | 0.5 | 0.166
0.1 | 0.6 | 0.17
0.1 | 1.0 | 0.18
0.2 | 0.2 | 0.2

在纯口碑数据集看来，召回最高也才 35% 左右，而预测数据集包括不少新用户。在提交第一次提交时，取的是 11 月的样本，召回率应该在 30% 以下，如果假设是 20% ，那么对应的准确率是 10% 。